In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda, num_samples=config.num_train_size)
# build dimension reduction model
hnne = get_hnne_model(X, y)
# get loader
loader = get_loader(X, y, hnne=hnne)
# get val loader
val_loader = get_val_loader(robot=panda, hnne=hnne)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth


In [3]:
mu = torch.randn(size=(7,))
mu

tensor([-0.7489,  1.0383, -0.5663,  0.4881,  1.2744, -0.3558, -0.9725])

In [4]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0.01)
# df, err = test_l2_err(robot=panda, 
#                       loader=val_loader, 
#                       model=iflow, 
#                       num_data=1, 
#                       num_samples=1000)
# # ax1 = df.plot.scatter(x='log_prob', y='l2_err')
# df.describe()

In [5]:
# df, err = test_l2_err(robot=panda, 
#                       loader=val_loader, 
#                       model=flow, 
#                       num_data=100, 
#                       num_samples=1000)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
# df.describe()

In [6]:
nflow = get_nflow_model(flow=flow)
# df, err = test_l2_err(robot=panda, 
#                       loader=val_loader, 
#                       model=nflow, 
#                       num_data=100, 
#                       num_samples=1000)
# # ax1 = df.plot.scatter(x='log_prob', y='l2_err')
# df.describe()

In [7]:
total_params = sum(
	param.numel() for param in flow.parameters()
)
print(total_params)

15964263


In [8]:
traj_dir = sample_ee_traj(robot=panda, load_time='')

mkdir /home/luca/ikpflow/data/panda/trajectory/0715103446
/home/luca/ikpflow/data/panda/trajectory/0715103446/ load successfully.


In [9]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=nflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

[[ 0.24924536 -0.59022633  0.86470717 -0.28445309]
 [-0.35049827 -0.19070481  0.33728153 -0.73179194]
 [-0.35045466 -0.19055979  0.33733271 -0.73197854]]
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.002257 -17.310195     155.761193
std     0.001083   0.956094     119.742300
min     0.000481 -18.699806      33.810536
25%     0.001667 -17.994552      56.663079
50%     0.002293 -17.610987     137.470636
75%     0.002754 -16.671510     216.189605
max     0.004562 -14.830688     467.334001
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.001838 -17.726886     133.917291
std     0.001012   0.563482      60.688763
min     0.000731 -18.710678      47.681035
25%     0.001175 -18.102830      80.527671
50%     0.001669 -17.787731     144.317675
75%     0.002236 -17.459035     173.696280
max     0.005150 -16.425322     241.549667
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000  

In [10]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=iflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

[[ 0.24813538 -0.58682241  0.86425244 -0.28480715]
 [ 0.24813167 -0.58682937  0.86423745 -0.28479792]
 [ 0.24813538 -0.58682241  0.86425244 -0.28480715]]
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.097284 -37.441664      56.705460
std     0.072696   3.081365      87.009142
min     0.013462 -43.080330       1.568568
25%     0.016767 -39.373061      16.960739
50%     0.106583 -37.686119      25.239227
75%     0.144081 -35.627348      58.636528
max     0.244529 -31.617817     387.820545
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.095424 -37.986368      58.339236
std     0.069687   2.931642      85.422793
min     0.013549 -43.500610       3.091420
25%     0.017128 -39.956064      16.900520
50%     0.106164 -37.481001      34.124855
75%     0.143885 -35.670015      57.781000
max     0.242130 -33.887199     384.319035
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000  

In [22]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

config.py: Using device 'cuda:0'


In [ ]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

In [ ]:
ee_traj = load_numpy(file_path=traj_dir + 'ee_traj.npy')
traj_dir, target_poses = get_target_poses(robot=panda)
exp_dir = traj_dir + f'exp_5_single_obj/'
data = {fn: [] for fn in file_names}

# -> unrefined solutions
l2_errs = np.zeros((num_ikflow_trails,))
ang_errs = np.zeros((num_ikflow_trails,))
for ikflow_i in range(num_ikflow_trails):
    solutions, l2_err, _, _, _, _ = ik_solver.solve_n_poses(
        target_poses, 
        refine_solutions=False, 
        return_detailed=True)

    iksols = solutions.detach().cpu().numpy()
    df = qtraj_evaluation(robot=panda, qs=iksols, l2_errs=l2_err)
    l2_errs[ikflow_i], ang_errs[ikflow_i] = df.mean().values
data['ikflow_l2'] = l2_errs
data['ikflow_ang'] = ang_errs    
